# GitHub Clones

In [ ]:
Home = '/kaggle/working/'
%cd {Home}
start = 0
#!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
#!git clone https://github.com/KeeganFernandesWork/yolo_tracking
if start:
    !pip install ultralytics
    !pip install -e detectron2_repo
    from ultralytics import YOLO

%cd {Home}
%cd yolo_tracking

if start:
    !pip install -r requirements.txt
    !pip install .

from IPython.display import clear_output
clear_output(wait=False)

# Libraries

In [ ]:
# Run after restart session
%matplotlib inline

import os
import cv2
import PIL
import sys
import time
import json
import torch
import shutil
import warnings
import detectron2
import numpy as np
import tracemalloc
import torch.nn as nn
from PIL import Image
from pathlib import Path
from ultralytics import YOLO
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
from detectron2 import model_zoo
from IPython.display import display
from detectron2.config import get_cfg
from detectron2.structures import Boxes
from IPython.display import clear_output
from torchvision import models, transforms
from detectron2.utils.logger import setup_logger
from sklearn.metrics.pairwise import cosine_similarity
from detectron2.data import build_detection_test_loader
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
warnings.filterwarnings("ignore", category=FutureWarning, message=".*torch.load.*weights_only=False.*")
from boxmot import (OCSORT, BoTSORT, BYTETracker, DeepOCSORT, StrongSORT, create_tracker, get_tracker_config)
setup_logger()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


ModuleNotFoundError: No module named 'detectron2'

# Functions

In [ ]:
# Define a CNN model with a variable input size control
class AdaptiveCNN(nn.Module):
    def __init__(self, output_size=(1, 1)):
        super(AdaptiveCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),  # Conv layer
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),                 # Downsample
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1), # Conv layer
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),                 # Downsample
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),# Conv layer
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),                 # Downsample
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1), # Conv layer
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(output_size)  # Adaptive pooling to ensure fixed-size output
        )
        self.flatten = nn.Flatten()  # Flatten to convert to 1D feature vector

    def forward(self, x):
        x = self.features(x)
        x = self.flatten(x)  # Flatten the feature map to a 1D feature vector
        return x


#==============================================================
def sift_feature_matching(target_img, crop_img):
    # Load the two images
    img1 = crop_img    # Query image
    img2 = target_img  # Train image
    # Convert images to grayscale
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # Initialize the SIFT detector
    sift = cv2.SIFT_create()

    # Detect SIFT keypoints and compute descriptors for both images
    keypoints1, descriptors1 = sift.detectAndCompute(img1, None)
    keypoints2, descriptors2 = sift.detectAndCompute(img2, None)

    # Initialize the BFMatcher
    bf = cv2.BFMatcher()

    # Use KNN matcher to find the best 2 matches for each descriptor
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)

    # Apply Lowe's ratio test to filter good matches
    good_matches = []
    for m, n in matches:
        if m.distance < 0.8 * n.distance:  # Ratio test
            good_matches.append(m)

    return len(good_matches)

In [ ]:
# Function to extract features from images or bounding boxes with controlled input size
def extract_features_variable_size(crop_img, model, input_size):
    # Resize the crop to the controlled input size
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((input_size, input_size)),  # Resize to the specified input size
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    crop_transformed = transform(crop_img).unsqueeze(0)

    # Extract features using the CNN model
    with torch.no_grad():
        features = model(crop_transformed)

    return features.squeeze().numpy()


#==============================================================
def extract_target_img(frame, target_bbox, show_crop_img=False):
    # Extract features for the target bounding box
    target_x1, target_y1, target_x2, target_y2 = target_bbox
    # Convert bounding box coordinates to integers
    target_x1 = int(target_x1)
    target_y1 = int(target_y1)
    target_x2 = int(target_x2)
    target_y2 = int(target_y2)
    target_crop = frame[target_y1:target_y2, target_x1:target_x2]
    if show_crop_img:
        plt.figure(figsize=(8, 6))  # Set the figure size
        plt.imshow(target_crop)

    return target_crop


#==============================================================
# Function to compute cosine similarity
def compute_similarity(features1, features2):
    return cosine_similarity(features1.reshape(1, -1), features2.reshape(1, -1))[0][0]


#==============================================================
# Eculidian Distance
def euclidean_distance(feature1, feature2):
    feature1 = np.array(feature1)
    feature2 = np.array(feature2)
    return np.linalg.norm(feature1 - feature2)


#==============================================================
def cv2_imshow(img, size=(8, 6), show=True, save='', title=''):
    if show:
        plt.figure(figsize=size)  # Set the figure size
        plt.axis('off')
        if len(title)>0:
            plt.title(title)

        plt.imshow(img)


    if os.path.isdir(save[:26]):
        plt.imsave(save, img)


#==============================================================
def draw_bounding_boxes(image, boxes, classes, gt, plot=False):
    # Draw ground truth bounding boxes
    if gt:
        color = (0, 0, 255)
    else:
        color = (0, 255, 0)

    item_no=1
    for box,clas in zip(boxes, classes):
        x_min, y_min, x_max, y_max = map(int, box)
        label = f'{item_no}__{clas}'
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color, 1)  # Blue for ground truth
        cv2.putText(image, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

        item_no +=1

    if(plot==True):
        plt.figure(figsize=(12, 8))
        plt.imshow(image)
        plt.title("Predicted Bounding Boxes and Classes")
        plt.axis('off')  # Hide axis
        plt.show()

    return image


#==============================================================
def xywhn_to_xyxy(cx, cy, w, h, image_width, image_height):
    """
    Convert YOLO format [cx, cy, w, h] with normalized values to [x_min, y_min, x_max, y_max] in pixel values.

    Parameters:
    - cx, cy: Center coordinates (normalized)
    - w, h: Width and height (normalized)
    - image_width: Width of the image in pixels
    - image_height: Height of the image in pixels

    Returns:
    - A list [x_min, y_min, x_max, y_max] in pixel values
    """
    x_min = (cx - w / 2) * image_width
    y_min = (cy - h / 2) * image_height
    x_max = (cx + w / 2) * image_width
    y_max = (cy + h / 2) * image_height
    return [x_min, y_min, x_max, y_max]


#==============================================================
def x1y1x2y2_to_xywh(box):
    """
    Converts bounding box from [x1, y1, x2, y2] to [x, y, w, h].

    Args:
        box (list or tuple): Bounding box in [x1, y1, x2, y2] format.

    Returns:
        list: Bounding box in [x, y, w, h] format.
    """
    x1, y1, x2, y2 = box
    x = x1
    y = y1
    w = x2 - x1
    h = y2 - y1
    return [x, y, w, h]


#==============================================================
def rescale_bboxes(pred_boxes):
    scaled_boxes = []
    for box in pred_boxes:
    # Example bounding box coordinates in the resized image (640x640)
        x_min_resized, y_min_resized, x_max_resized, y_max_resized = box

        # Original image dimensions
        original_width = 1920
        original_height = 1080

        # Resized image dimensions
        resized_width = 640
        resized_height = 640

        # Scaling factors
        width_scaling_factor = original_width / resized_width
        height_scaling_factor = original_height / resized_height

        # Scale bounding box coordinates
        x_min_original = x_min_resized * width_scaling_factor
        y_min_original = y_min_resized * height_scaling_factor
        x_max_original = x_max_resized * width_scaling_factor
        y_max_original = y_max_resized * height_scaling_factor

        # Output scaled bounding box
        scaled_boxes.append( (x_min_original, y_min_original, x_max_original, y_max_original) )

    return scaled_boxes

In [ ]:
# Function to filter bounding boxes based on feature similarity
def filter_bboxes(frame, bboxes, target_img, threshold=0.8, show_filter_img=False, show_scores=False):
    similarity_scores =[]
    sift_scores = []

    # Iterate through all bounding boxes
    for bbox in bboxes:
        # Extract features for the current bounding box
        x1, y1, x2, y2 = bbox
        x1 = int(x1)
        y1 = int(y1)
        x2 = int(x2)
        y2 = int(y2)
        crop = frame[y1:y2, x1:x2]
        if crop.size == 0:  # Skip empty crops
            continue

        min_input_size = crop.shape[0] if crop.shape[0] < crop.shape[1] else crop.shape[1]
        if min_input_size < 16:
            continue

        # Initialize the CNN model with adaptive pooling to a fixed output size
        model = AdaptiveCNN(output_size=(1, 1))  # Output size after pooling
        model.eval()  # Set the model to evaluation mode


        crop = cv2.resize(crop, (min_input_size, min_input_size))
        # Extract features from the bounding boxes with controlled input size
        target_features = extract_features_variable_size(target_img.copy(), model, min_input_size)
        detection_features = extract_features_variable_size(crop, model, min_input_size)

        # Compute similarity between the two feature vectors
        sift_scores.append(sift_feature_matching(target_img.copy(), crop))
        similarity_scores.append(compute_similarity(target_features, detection_features))

    if show_filter_img:
        frame_copy = frame.copy()
        i=1
        for bbox in bboxes:
            x1, y1, x2, y2 = bbox
            x1 = int(x1)
            y1 = int(y1)
            x2 = int(x2)
            y2 = int(y2)
            cv2.rectangle(frame_copy, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame_copy, f"BBox{i}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            i+=1

        cv2_imshow(frame_copy)


    try:
        max_similarity_indexes = [index for index, value in enumerate(similarity_scores) if value > 0.9]
        possible_target_sift_values = [sift_scores[i] for i in max_similarity_indexes]

        seen = set()
        duplicates = set(x for x in possible_target_sift_values if x in seen or seen.add(x)) # find duplicates
        if (duplicates):
            max_final_index = np.argmax(similarity_scores)
        else:
            max_sift_index = np.argmax(possible_target_sift_values)
            max_final_index = max_similarity_indexes[max_sift_index]
    except:
        max_final_index = -1


    if show_scores:
        print('\n\n')
        print('Sift_scores:', sift_scores)
        print('Resnet Simlarity scores:', similarity_scores)
        print('Max Index:', max_final_index)


    return max_final_index

In [ ]:
Home = '/kaggle/working/'
color = (0, 255, 0)  # BGR
thickness = 2
fontscale = 1

device = "cuda" # cuda:0 , cpu
fp16 = True # True if gpu available

source = '/kaggle/input/test-video-15fps/Test4_15fps.mp4'
target_label = '/kaggle/input/test-dataset/test/labels/frame_0001.txt'
first_frame = '/kaggle/input/test-dataset/test/images/frame_0001.jpg'
wgts_dir = '/kaggle/input/all-weights/'

model_names = ['faster_rcnn', 'retinanet', 'cascade_rcnn', 'yolo']
all_wgts = ['faster_rcnn.pth', 'retinaNet.pth', 'cascade_rcnn.pth', 'yolo.pt']
detectron_models = ["COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml", "COCO-Detection/retinanet_R_50_FPN_1x.yaml", "Misc/cascade_mask_rcnn_R_50_FPN_1x.yaml"]

In [ ]:
def setup_config(model, output_dir):
    cfg = get_cfg()

    cfg.merge_from_file(model_zoo.get_config_file(model))
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model)
    #cfg.DATASETS.TRAIN = (train_data,)
    #cfg.DATASETS.TEST = (val_data,)
    cfg.TEST.EVAL_PERIOD = 500
    cfg.DATALOADER.NUM_WORKERS = 2
    cfg.SOLVER.IMS_PER_BATCH = 16
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  # Number of classes for Faster RCNN
    cfg.MODEL.RETINANET.NUM_CLASSES = 3  # Number of classes for Retina Net
    cfg.MODEL.MASK_ON = False
    cfg.MODEL.LOAD_PROPOSALS = False

    # Debug/verbose
    setup_logger()
    # Create the output directory
    cfg.OUTPUT_DIR = output_dir

    return cfg


#==============================================================
def load_yolo(image, weight, threshold):
    start_time = time.time()
    tracemalloc.start()
    #-----------------------
    image = cv2.resize(image, (640,640))

    model = YOLO(weight)
    results = model.predict(image, conf=threshold)
    pred_boxes = results[0].boxes.xyxy.cpu().numpy()  # Predicted bounding boxes
    pred_classes = results[0].boxes.data[:, 5].cpu().numpy()  # Predicted class IDs
    pred_scores = results[0].boxes.data[:, 4].cpu().numpy()  # Confidence scores
    #------------------------
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    end_time = time.time()
    execution_time = end_time - start_time
    execution_time = np.round(execution_time,2)

    pred_boxes = rescale_bboxes(pred_boxes)
    return pred_boxes, pred_classes, pred_scores, execution_time


#==============================================================
def load_detectron2(image, yaml_model, threshold, all_models=detectron_models, all_wgts=all_wgts, wgts_dir = wgts_dir):
    start_time = time.time()
    tracemalloc.start()
    #------------------------------------
    image = cv2.resize(image, (640,640))

    model_index = all_models.index(yaml_model)
    cfg = setup_config(yaml_model, wgts_dir)
    cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, all_wgts[model_index])

    # Step 2: Create a Predictor
    predictor = DefaultPredictor(cfg)
    outputs = predictor(image)

    custom_threshold = threshold
    instances = outputs["instances"]

    # Extract the bounding boxes, class labels, and scores
    boxes = instances.pred_boxes if instances.has("pred_boxes") else None
    scores = instances.scores if instances.has("scores") else None
    classes = instances.pred_classes if instances.has("pred_classes") else None

    # Filter predictions based on the custom threshold
    filtered_indices = scores > custom_threshold
    filtered_boxes = boxes[filtered_indices]
    filtered_scores = scores[filtered_indices]
    filtered_classes = classes[filtered_indices]

    pred_boxes = filtered_boxes.tensor.to('cpu').tolist()
    pred_scores = filtered_scores.to('cpu').tolist()
    pred_classes = filtered_classes.to('cpu').tolist()
    #-------------------------------------------------
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    end_time = time.time()
    execution_time = end_time - start_time
    execution_time = np.round(execution_time,2)

    pred_boxes = rescale_bboxes(pred_boxes)
    return pred_boxes, pred_classes, pred_scores, execution_time


#==============================================================
def create_video_writer(video_cap, output_filename):

    # grab the width, height, and fps of the frames in the video stream.
    frame_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_cap.get(cv2.CAP_PROP_FPS))

    # initialize the FourCC and a video writer object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    writer = cv2.VideoWriter(output_filename, fourcc, fps,
                             (frame_width, frame_height))

    return writer

# All Trackers

In [ ]:
tracker_DeepOCSORT = DeepOCSORT(
    model_weights=Path('osnet_x0_25_msmt17.pt'), # which ReID model to use
    device=device,
    fp16=fp16,
    det_thresh=0.1,      # Only detections with a confidence score above this threshold are considered valid and are passed to the tracking algorithm.
    max_age=50,          # Specifies the maximum number of frames a tracker can go without being updated by a new detection before it is considered lost.
    min_hits=3,          # Defines the minimum number of hits (consecutive frames with detections) needed to consider a track as confirmed.
    iou_threshold=0.3,   # The Intersection over Union (IoU) threshold used for associating detections with existing tracks.
)

tracker_StrongSORT = StrongSORT(
    model_weights=Path('mobilenetv2_x1_4_dukemtmcreid.pt'), # which ReID model to use
    device=device,
    fp16=fp16,
    max_dist=0.1,             # Sets the maximum allowable distance (typically a cosine distance) between appearance features for matching detections to existing tracks.
    max_iou_dist=0.3,         # Specifies the maximum Intersection over Union (IoU) distance allowed for associating detections with predicted tracks.
    max_age=50,               # Defines the maximum number of frames a track can exist without being updated by a new detection before being considered lost.
    max_unmatched_preds=7,    # Specifies the maximum number of unmatched predictions that can be retained.
    n_init=3,                 # Sets the number of consecutive frames in which a detection must be associated with a track before it is confirmed.
    nn_budget=100,            # Limits the number of appearance features stored for each track.
    mc_lambda=0.995,          # A parameter related to motion compensation; it controls the decay rate of the motion model.
    ema_alpha=0.9             # Sets the alpha value for the Exponential Moving Average (EMA) used to smooth the appearance features of a track.
)

tracker_BoTSORT = BoTSORT(
    model_weights=Path('osnet_x0_25_msmt17.pt'),  # Update with the correct model path
    device='cuda',  # or 'cuda' if using GPU
    fp16=False,  # Set to True if using FP16 precision
    track_high_thresh=0.1,
    new_track_thresh=0.3,
    track_buffer=50,
    match_thresh=0.3,
    frame_rate=15
)

tracker_BYTETracker =  BYTETracker(
    track_thresh=0.1,  # Sets the threshold for the minimum confidence score required for a detection to be considered for tracking.
    match_thresh=0.3,  # Specifies the threshold for matching existing tracks with new detections based on IoU (Intersection over Union) distance.
    track_buffer=50,   # Determines the maximum number of frames that a track can remain without being updated by a new detection before being considered lost.
    det_thresh =0.1,
    frame_rate=15      # The frame rate of the input video, typically specified in frames per second (fps).
)

trackers_list = [tracker_DeepOCSORT, tracker_StrongSORT, tracker_BoTSORT, tracker_BYTETracker]
trackers_names = ['DeepOCSORT', 'StrongSORT', 'BotSORT', 'ByteTracker']
thresholds = [0.5, 0.3, 0.1, 1e-5]

2024-10-01 05:30:36.046 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:187 - Successfully loaded pretrained weights from "osnet_x0_25_msmt17.pt"
2024-10-01 05:30:36.047 | WARNING  | boxmot.appearance.reid_model_factory:load_pretrained_weights:191 - The following layers are discarded due to unmatched keys or layer size: ('classifier.weight', 'classifier.bias')
2024-10-01 05:30:36.342 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:187 - Successfully loaded pretrained weights from "mobilenetv2_x1_4_dukemtmcreid.pt"
2024-10-01 05:30:36.344 | WARNING  | boxmot.appearance.reid_model_factory:load_pretrained_weights:191 - The following layers are discarded due to unmatched keys or layer size: ('classifier.weight', 'classifier.bias')
2024-10-01 05:30:36.448 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:187 - Successfully loaded pretrained weights from "osnet_x0_25_msmt17.pt"
2024-10-01 05:30:36.449 | WARNING  | bo

In [ ]:
for tracker_name in trackers_names:
    tracker = trackers_list[trackers_names.index(tracker_name)]
    out_dir = f'{Home}/Output/{tracker_name}/'
    os.makedirs(out_dir, exist_ok=True)

    for model_name in model_names:
        for threshold in thresholds:
            # ===================================================================
            tracker = BYTETracker(frame_rate=15)
            start_time = time.time()
            tracemalloc.start()
            # ===================================================================
            # Extract ground truth
            image = cv2.imread(first_frame)
            with open(target_label, 'r') as f:
                lines = f.readlines()

            target_gt = lines[0].split()
            target_gt = [float(x) for x in target_gt]  # convert string to float
            target_bbox = xywhn_to_xyxy(target_gt[1], target_gt[2], target_gt[3], target_gt[4], image_width=image.shape[1], image_height=image.shape[0])
            target_img = extract_target_img(image, target_bbox)

            #=========================================================================
            vid = cv2.VideoCapture(source)
            #os.remove(out_dir + tracker_name +'.mp4') # delete old ones
            #writer = create_video_writer(vid, out_dir + f'{tracker_name}_{model_name}_{threshold}.mp4')

            tracker_predictions = []
            frame_no = 0

            show = False
            debug = []
            lets_debug = 0
            frame_debug_start = 0
            frame_debug_end = 0

            while True:
                ret, frame = vid.read()
                #=========================================================================
                if(lets_debug == 1):
                    if(frame_no >frame_debug_start and frame_no < frame_debug_end):
                        show=True
                    elif(frame_no > frame_debug_end):
                        break
                    else:
                        frame_no +=1
                        continue
                        clear_output(wait=False)
                else:
                    clear_output(wait=False)

                #========================================================================

              # if video frames end
                if not ret:
                    break
                else:
                    frame_no += 1
                    print(f'Tracker:{tracker_name}  Model:{model_name}  Threshold:{threshold}  Frame No:{frame_no}')

                if model_name == 'yolo':
                    weight = wgts_dir + all_wgts[model_names.index(model_name)]
                    pred_boxes, pred_classes, pred_scores, execution_time = load_yolo(frame.copy(), weight, threshold)
                else:
                    yaml_model = detectron_models[model_names.index(model_name)]
                    pred_boxes, pred_classes, pred_scores, execution_time = load_detectron2(frame.copy(), yaml_model, threshold)


                filtered_index = filter_bboxes(frame.copy(), pred_boxes, target_img, threshold=0.8, show_filter_img=show, show_scores=show)
                if filtered_index == -1:
                    dets = np.empty((0, 6))
                else:
                    dets = list(pred_boxes[filtered_index])
                    dets.append(pred_scores[filtered_index])
                    dets.append(pred_classes[filtered_index])
                    dets = np.array([dets])

                ts = tracker.update(dets, frame.copy()) # --> ts output = (x1, y1, x2, y2, id, conf, cls)
                if show:
                    print('Filter Index:', filtered_index)
                    print('Detection:', dets)
                    print('Tracker Output:', ts)

                xyxys = ts[:,0:4].astype('int') # float64 to int
                ids = ts[:, 4].astype('int') # float64 to int
                confs = np.round(ts[:, 5], 2)
                clss = ts[:, 6]

              # print bboxes with their associated id, cls and conf
                for xyxy, id, conf, cls in zip(xyxys, ids, confs, clss):
                    im = cv2.rectangle(
                    frame,
                    (xyxy[0], xyxy[1]),
                    (xyxy[2], xyxy[3]),
                    color,
                    thickness
                    )

                    cv2.putText(
                    frame,
                    f'id: {id}, conf: {conf}, c: {cls}',
                    (xyxy[0], xyxy[1]-10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    fontscale,
                    color,
                    thickness
                    )
                    break

                height, width = image.shape[:2]
                text = f'Frame: {frame_no}'
                position = (width - 300, 30)
                cv2.putText(frame, text, position, cv2.FONT_HERSHEY_SIMPLEX, fontscale, color, thickness)
                # save tracker predictions for benchmark
                # <frame_no>, <tracke_obj_id>, <bb_left>, <bb_top>, <bb_width>, <bb_height>, <conf>, -1 -1 -1
                # <frame_no>, <tracke_obj_id>, <x1>, <y1>, <x2-x1>, <y2-y1>, <conf>, -1 -1 -1
                try:
                    pred = f"{frame_no}, {id}, {xyxy[0]}, {xyxy[1]}, {xyxy[2]-xyxy[0]}, {xyxy[2]-xyxy[0]}, {conf}, -1, -1, -1"
                except:
                    xyxy = [-1,-1,-1,-1]
                    id = -1
                    conf = -1
                    pred = f"{frame_no}, {id}, {xyxy[0]}, {xyxy[1]}, {xyxy[2]-xyxy[0]}, {xyxy[2]-xyxy[0]}, {conf}, -1, -1, -1"

                tracker_predictions.append(pred)

                #debug_txt = f'{frame_no}, Dets:{dets}, Ts:{ts}'
                #debug.append(debug_txt)

                if show:
                    cv2_imshow(frame, title=f'{frame_no}')

                #writer.write(frame)


            #vid.release()
            #writer.release()
            # ===================================================================
            current, peak = tracemalloc.get_traced_memory()
            tracemalloc.stop()

            # Calculate the time taken
            end_time = time.time()
            execution_time = end_time - start_time

            Execution_time = np.round(execution_time,2)
            Current_memory = np.round(current / 1024 / 1024, 2)
            Peak_memory = np.round(peak / 1024 / 1024, 2)

            simulation_performance = [Execution_time, Current_memory, Peak_memory]
            # ===================================================================
            with open(out_dir + f'{tracker_name}_{model_name}_{threshold}_pred.txt', 'w') as mot_file:
                for pred in tracker_predictions:
                        mot_file.write(pred + '\n')


            with open(out_dir + f'{tracker_name}_{model_name}_{threshold}_simulation_performance.txt', 'w') as file:
                    file.write(f'Execution_time \t Current_memory \t Peak_memory \n')
                    file.write(f'{Execution_time} \t {Current_memory} \t {Peak_memory} \n')


print("Task Completed")

Task Completed


# Edit python files

In [ ]:
# Edit .py file in GitHub and Upload to Kaggle

import shutil
import os

Home = '/kaggle/working/'
shutil.rmtree('/kaggle/working/Random-Custom-Functions')

%cd {Home}
!git clone https://github.com/tasnemul-hasan/Random-Custom-Functions.git

src_list=['deep_ocsort.py', 'strong_sort.py', 'bot_sort.py', 'byte_tracker.py']
dest_trsckers = ['deepocsort', 'strongsort', 'botsort', 'bytetrack']

for i in range(4):
    # Define source and destination paths
    source_path = f'/kaggle/working/Random-Custom-Functions/YoloTracking/{src_list[i]}'
    destination_path = f'/kaggle/working/yolo_tracking/boxmot/trackers/{dest_trsckers[i]}/{src_list[i]}'

    os.remove(destination_path)
    shutil.copyfile(source_path, destination_path)

    print(destination_path)

/kaggle/working
Cloning into 'Random-Custom-Functions'...
remote: Enumerating objects: 224, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 224 (delta 9), reused 1 (delta 1), pack-reused 200 (from 1)
Receiving objects: 100% (224/224), 75.96 KiB | 1.19 MiB/s, done.
Resolving deltas: 100% (61/61), done.
/kaggle/working/yolo_tracking/boxmot/trackers/deepocsort/deep_ocsort.py
/kaggle/working/yolo_tracking/boxmot/trackers/strongsort/strong_sort.py
/kaggle/working/yolo_tracking/boxmot/trackers/botsort/bot_sort.py
/kaggle/working/yolo_tracking/boxmot/trackers/bytetrack/byte_tracker.py


"\n# Make Zip to download all results\n\nOUTPUT_NAME = f'/kaggle/working/Tracker_Output'\nDIRECTORY_TO_ZIP = f'/kaggle/working/Output'\n\nprint(DIRECTORY_TO_ZIP)\nshutil.make_archive(OUTPUT_NAME, 'zip', DIRECTORY_TO_ZIP)\n"

# Tracker Output Modification

In [ ]:
# Replace the second column of the DataFrame with the modified list
import pandas as pd

model_names = ['faster_rcnn', 'retinanet', 'cascade_rcnn', 'yolo']
trackers_names = ['DeepOCSORT', 'StrongSORT', 'BotSORT', 'ByteTracker']
thresholds = [0.5, 0.3, 0.1, 1e-5]

for tracker_name in trackers_names:
    print(tracker_name)
    for model_name in model_names:
        for threshold in thresholds:
            # Load the data from the text file into a DataFrame
            dir_name = f'/kaggle/working/Output/{tracker_name}/'
            file_name = f'{tracker_name}_{model_name}_{threshold}_pred.txt'
            data = pd.read_csv(dir_name + file_name, header=None)

            # Extract the second column as a list
            second_column = data[1].tolist()

            # Subtract all values in the list from first_ID then add 1
            first_ID = second_column[0]
            modified_list = [(abs(first_ID-value)+1) for value in second_column]

            data[1] = modified_list

            # Save the modified DataFrame to a new text file
            output_file = dir_name + 'Mod_'+ file_name
            data.to_csv(output_file, header=False, index=False, sep=',')

DeepOCSORT
StrongSORT
BotSORT
ByteTracker


# Tracker Evaluation

In [ ]:
import shutil
import os

model_names = ['faster_rcnn', 'retinanet', 'cascade_rcnn', 'yolo']
trackers_names = ['DeepOCSORT', 'StrongSORT', 'BotSORT', 'ByteTracker']
thresholds = [0.5, 0.3, 0.1, 1e-5]

dest_folder = '/kaggle/working/TrackEval/data/trackers/mot_challenge/TestVideo4-train/'
src_tracker_folder = '/kaggle/working/TrackEval/data/trackers/mot_challenge/TestVideo4-train/PerfectTracker'


for tracker_name in trackers_names:
    print(tracker_name)
    for model_name in model_names:
        for threshold in thresholds:
            # Load the data from the text file into a DataFrame
            new_folder_name = f'{tracker_name}_{model_name}_{threshold}'
            dest_tracker_folder = dest_folder+new_folder_name
            if os.path.exists(dest_tracker_folder):
                # Remove the destination folder if it exists
                shutil.rmtree(dest_tracker_folder)
            shutil.copytree(src_tracker_folder, dest_tracker_folder)

            src_txt_dir = f'/kaggle/working/Output/{tracker_name}/'
            src_txt_file_name = src_txt_dir + f'Mod_{tracker_name}_{model_name}_{threshold}_pred.txt'
            dest_txt_file_name = dest_tracker_folder + '/data/TestVideo4-01.txt'
            shutil.copyfile(src_txt_file_name, dest_txt_file_name)

DeepOCSORT
StrongSORT
BotSORT
ByteTracker


In [ ]:
%cd /kaggle/working/TrackEval

!python scripts/run_mot_challenge.py --BENCHMARK TestVideo4 --SPLIT_TO_EVAL train --METRICS HOTA CLEAR Identity VACE --USE_PARALLEL False --NUM_PARALLEL_CORES 1


/kaggle/working/TrackEval
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Error importing BURST due to missing underlying dependency: No module named 'pycocotools'

Eval Config:
USE_PARALLEL         : False                         
NUM_PARALLEL_CORES   : 1                             
BREAK_ON_ERROR       : True                          
RETURN_ON_ERROR      : False                         
LOG_ON_ERROR         : /kaggle/working/TrackEval/error_log.txt
PRINT_RESULTS        : True                          
PRINT_ONLY_COMBINED  : False                         
PRINT_CONFIG         : True                          
TIME_PROGRESS        : True                          
DISPLAY_LESS_PROGRESS : False                         
OUTPUT_SUMMARY       : True                          
OUTPUT_EMPTY_CLA

In [ ]:
import pandas as pd
import numpy as np

def filtered_dataframe(name, track_dir, target_columns):
    csv_file = track_dir + name + '/target_vehicle_detailed.csv'
    df = pd.read_csv(csv_file)

    df_filtered = df[target_columns]
    df_filtered = df_filtered.iloc[:-1]   # remove last row
    df_filtered['Tracker'] = name # add tracker name column
    df_filtered = df_filtered.rename(columns={'HOTA___AUC': 'HOTA_AUC'})

    # Move the last column ('Tracker') to the first position
    columns = df_filtered.columns.tolist()  # Get the list of columns
    new_order = [columns[-1]] + columns[:-1]  # Move the last column to the front
    df_filtered = df_filtered[new_order]
    return df_filtered

track_dir = '/kaggle/working/TrackEval/data/trackers/mot_challenge/TestVideo4-train/'
trackers_names = ['DeepOCSORT', 'StrongSORT', 'BotSORT', 'ByteTracker']
model_names = ['faster_rcnn', 'retinanet', 'cascade_rcnn', 'yolo']
thresholds = [0.5, 0.3, 0.1, 1e-5]


target_columns = ['HOTA___AUC', 'MOTP', 'CLR_TP', 'CLR_FP', 'IDSW', 'IDF1', 'IDR', 'IDP']

df_list = []
for tracker_name in trackers_names:
    print(tracker_name)
    for model_name in model_names:
        for threshold in thresholds:
            name = f'{tracker_name}_{model_name}_{threshold}'
            df_list.append(filtered_dataframe(name, track_dir, target_columns).copy())


df_list.append(filtered_dataframe('PerfectTracker', track_dir, target_columns))
df_combined = pd.concat(df_list, axis=0, ignore_index=True)
df_combined.to_csv('/kaggle/working/Output/All_tracker_output.csv', index=False, sep=',')
df_combined

DeepOCSORT
StrongSORT
BotSORT
ByteTracker


,Tracker,HOTA_AUC,MOTP,CLR_TP,CLR_FP,IDSW,IDF1,IDR,IDP
0,DeepOCSORT_faster_rcnn_0.5,0.306819,0.646812,642,333,2,0.466598,0.467559,0.465641
1,DeepOCSORT_faster_rcnn_0.3,0.275749,0.662665,532,443,4,0.448099,0.449022,0.447179
2,DeepOCSORT_faster_rcnn_0.1,0.234278,0.663360,415,560,6,0.365879,0.366632,0.365128
3,DeepOCSORT_faster_rcnn_1e-05,0.131560,0.675665,333,642,6,0.198356,0.198764,0.197949
4,DeepOCSORT_retinanet_0.5,0.209445,0.641987,494,481,1,0.304214,0.304840,0.303590
...,...,...,...,...,...,...,...,...,...
60,ByteTracker_yolo_0.5,0.328323,0.652223,706,269,4,0.496403,0.497425,0.495385
61,ByteTracker_yolo_0.3,0.326642,0.653807,703,272,3,0.494347,0.495366,0.493333
62,ByteTracker_yolo_0.1,0.331534,0.655399,709,266,3,0.493320,0.494336,0.492308
63,ByteTracker_yolo_1e-05,0.072981,0.657092,398,577,32,0.059609,0.059732,0.059487


# Convert CSV for Statical Analysis

In [ ]:
# In static analysis, lowest value is considered the best value. So observing the perfect tracker/ground truth, we will convert certain column
# Multiply both 'Age' and 'Salary' columns by -1
df_combined_statics = df_combined.copy()

selected_column = ['HOTA_AUC', 'MOTP', 'CLR_TP', 'IDF1', 'IDR', 'IDP']

for column in selected_column:
    df_combined_statics[column] = df_combined_statics[column] * -1

df_combined_statics.to_csv('/kaggle/working/Output/All_tracker_output_statics.csv', index=False, sep=',')
df_combined_statics

,Tracker,HOTA_AUC,MOTP,CLR_TP,CLR_FP,IDSW,IDF1,IDR,IDP
0,DeepOCSORT_faster_rcnn_0.5,-0.306819,-0.646812,-642,333,2,-0.466598,-0.467559,-0.465641
1,DeepOCSORT_faster_rcnn_0.3,-0.275749,-0.662665,-532,443,4,-0.448099,-0.449022,-0.447179
2,DeepOCSORT_faster_rcnn_0.1,-0.234278,-0.663360,-415,560,6,-0.365879,-0.366632,-0.365128
3,DeepOCSORT_faster_rcnn_1e-05,-0.131560,-0.675665,-333,642,6,-0.198356,-0.198764,-0.197949
4,DeepOCSORT_retinanet_0.5,-0.209445,-0.641987,-494,481,1,-0.304214,-0.304840,-0.303590
...,...,...,...,...,...,...,...,...,...
60,ByteTracker_yolo_0.5,-0.328323,-0.652223,-706,269,4,-0.496403,-0.497425,-0.495385
61,ByteTracker_yolo_0.3,-0.326642,-0.653807,-703,272,3,-0.494347,-0.495366,-0.493333
62,ByteTracker_yolo_0.1,-0.331534,-0.655399,-709,266,3,-0.493320,-0.494336,-0.492308
63,ByteTracker_yolo_1e-05,-0.072981,-0.657092,-398,577,32,-0.059609,-0.059732,-0.059487


In [ ]:
df_combined_statics_transposed = df_combined_statics.copy().T
df_combined_statics_transposed = df_combined_statics_transposed.reset_index()
df_combined_statics_transposed.to_csv('/kaggle/working/Output/All_tracker_output_statics_transposed.csv', index=False, sep=',')
df_combined_statics_transposed

,index,0,1,2,3,4,5,6,7,8,...,55,56,57,58,59,60,61,62,63,64
0,Tracker,DeepOCSORT_faster_rcnn_0.5,DeepOCSORT_faster_rcnn_0.3,DeepOCSORT_faster_rcnn_0.1,DeepOCSORT_faster_rcnn_1e-05,DeepOCSORT_retinanet_0.5,DeepOCSORT_retinanet_0.3,DeepOCSORT_retinanet_0.1,DeepOCSORT_retinanet_1e-05,DeepOCSORT_cascade_rcnn_0.5,...,ByteTracker_retinanet_1e-05,ByteTracker_cascade_rcnn_0.5,ByteTracker_cascade_rcnn_0.3,ByteTracker_cascade_rcnn_0.1,ByteTracker_cascade_rcnn_1e-05,ByteTracker_yolo_0.5,ByteTracker_yolo_0.3,ByteTracker_yolo_0.1,ByteTracker_yolo_1e-05,PerfectTracker
1,HOTA_AUC,-0.306819,-0.275749,-0.234278,-0.13156,-0.209445,-0.228014,-0.067758,-0.058115,-0.252914,...,-0.071609,-0.233451,-0.218935,-0.146942,-0.070198,-0.328323,-0.326642,-0.331534,-0.072981,-1.0
2,MOTP,-0.646812,-0.662665,-0.66336,-0.675665,-0.641987,-0.664427,-0.654996,-0.646243,-0.661213,...,-0.658893,-0.671667,-0.674764,-0.682931,-0.668791,-0.652223,-0.653807,-0.655399,-0.657092,-1.0
3,CLR_TP,-642,-532,-415,-333,-494,-578,-247,-93,-612,...,-81,-579,-611,-456,-437,-706,-703,-709,-398,-971
4,CLR_FP,333,443,560,642,481,397,728,882,363,...,894,396,364,519,538,269,272,266,577,0
5,IDSW,2,4,6,6,1,4,8,3,3,...,5,16,20,29,53,4,3,3,32,0
6,IDF1,-0.466598,-0.448099,-0.365879,-0.198356,-0.304214,-0.321686,-0.053443,-0.065776,-0.373073,...,-0.057554,-0.36999,-0.327852,-0.215827,-0.062693,-0.496403,-0.494347,-0.49332,-0.059609,-1.0
7,IDR,-0.467559,-0.449022,-0.366632,-0.198764,-0.30484,-0.322348,-0.053553,-0.065911,-0.373841,...,-0.057673,-0.370752,-0.328527,-0.216272,-0.062822,-0.497425,-0.495366,-0.494336,-0.059732,-1.0
8,IDP,-0.465641,-0.447179,-0.365128,-0.197949,-0.30359,-0.321026,-0.053333,-0.065641,-0.372308,...,-0.057436,-0.369231,-0.327179,-0.215385,-0.062564,-0.495385,-0.493333,-0.492308,-0.059487,-1.0


In [ ]:
df_combined_statics_transposed.to_csv('/kaggle/working/Output/All_tracker_output_statics_transposed.txt', sep=',', index=False)

In [ ]:
import shutil
# Make Zip to download all results

OUTPUT_NAME = f'/kaggle/working/Tracker_Output'
DIRECTORY_TO_ZIP = f'/kaggle/working/Output'

print(DIRECTORY_TO_ZIP)
shutil.make_archive(OUTPUT_NAME, 'zip', DIRECTORY_TO_ZIP)


/kaggle/working/Output


'/kaggle/working/Tracker_Output.zip'